Chúng ta sẽ xây dựng mô hình Text Classification sử dụng Bi-LSTM

dữ liệu chính là bộ VNTC, dữ liệu có hai tập train và test. Do tập test lớn hơn nên sẽ tạm thời dùng bộ test, từ đây chia ra các tập train và test. Số lớp cần phân loại là 10. 

Trước tiên chúng ta cần tải và giải nén dữ liệu.

In [1]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1oUV7CDhXVL5QVYSRLm9q_dxihaqSOgDr' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1oUV7CDhXVL5QVYSRLm9q_dxihaqSOgDr" -O Test_Full.rar && rm -rf /tmp/cookies.txt

--2020-09-10 12:52:29--  https://docs.google.com/uc?export=download&confirm=9m6e&id=1oUV7CDhXVL5QVYSRLm9q_dxihaqSOgDr
Resolving docs.google.com (docs.google.com)... 173.194.69.138, 173.194.69.101, 173.194.69.100, ...
Connecting to docs.google.com (docs.google.com)|173.194.69.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0c-4c-docs.googleusercontent.com/docs/securesc/g5f33s49udnfmo1b5jq9gbu3ivhhdjmg/1pjk3k9lli692pjb5gd2184biqrcfcvf/1599742275000/04205927457588895310/11278178322627907671Z/1oUV7CDhXVL5QVYSRLm9q_dxihaqSOgDr?e=download [following]
--2020-09-10 12:52:30--  https://doc-0c-4c-docs.googleusercontent.com/docs/securesc/g5f33s49udnfmo1b5jq9gbu3ivhhdjmg/1pjk3k9lli692pjb5gd2184biqrcfcvf/1599742275000/04205927457588895310/11278178322627907671Z/1oUV7CDhXVL5QVYSRLm9q_dxihaqSOgDr?e=download
Resolving doc-0c-4c-docs.googleusercontent.com (doc-0c-4c-docs.googleusercontent.com)... 108.177.127.132, 2a00:1450:4013:c07::84
Connectin

In [2]:
! ls

sample_data  Test_Full.rar


In [3]:
! unrar x /content/Test_Full.rar

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Extracting  Test_Full/Van hoa/VH_VNE_T_ (6210).txt                        89%  OK 
Extracting  Test_Full/Van hoa/VH_VNE_T_ (6079).txt                        89%  OK 
Extracting  Test_Full/Van hoa/VH_VNE_T_ (6083).txt                        89%  OK 
Extracting  Test_Full/Van hoa/VH_VNE_T_ (6086).txt                        89%  OK 
Extracting  Test_Full/Van hoa/VH_VNE_T_ (6088).txt                        89%  OK 
Extracting  Test_Full/Van hoa/VH_VNE_T_ (6312).txt                        89%  OK 
Extracting  Test_Full/Van hoa/VH_VNE_T_ (6092).txt                        89%  OK 
Extracting  Test_Full/Van hoa/VH_VNE_T_ (6094).txt                        89%  OK 
Extracting  Test_Full/Van hoa/VH_VNE_T_ (6101).txt                        89%  OK 
Extracting  Test_Full/Van hoa/VH_VNE_T_ (6108).txt                        89%  OK 
Extracting  Test_Full/V

Tiến hành đọc dữ liệu

In [4]:
import os
import time
from os.path import join
import string
import re
import random

In [5]:
data_path = '/content/Test_Full'

In [6]:
dataset = dict()

for category in os.listdir(data_path):
  dataset[category] = list()
  for file in os.listdir(join(data_path, category)):
    content = open(join(data_path, category, file), 'rb').read().decode('utf-16').strip()
    # Xóa dấu câu
    content = content.translate(str.maketrans('', '', string.punctuation))
    # Xóa số 
    content = re.sub(r'\d+', '', content)
    # Xử lý các noise trong dữ liệu, các kí tự \r\n
    content = content.replace('\r', '')
    content = content.replace('\n', ' ')
    dataset[category].append(content)

## Tách từ trong câu 

Bài toán tokenizer (tách từ) là một bài toán quan trọng, việc tách từ cho tiếng Việt tương đối khó và có chút đặc biệt khi mà số lượng từ ghép cho tiếng Việt rất nhiều. 

Chúng ta sẽ sử dụng thư viện tokenizer mà Cốc Cốc đã public. 

Việc cài đặt sẽ có chút đặc biệt. 


In [7]:
! sudo apt-get install python3 python-dev python3-dev \
     build-essential libssl-dev libffi-dev \
     libxml2-dev libxslt1-dev zlib1g-dev \
     python-pip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
python-dev is already the newest version (2.7.15~rc1-1).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
libssl-dev is already the newest version (1.1.1-1ubuntu2.1~18.04.6).
libxml2-dev is already the newest version (2.9.4+dfsg1-6.1ubuntu1.3).
python3 is already the newest version (3.6.7-1~18.04).
python3-dev is already the newest version (3.6.7-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libpython-all-dev python-all python-all-dev python-asn1crypto
  python-cffi-backend python-crypto python-cryptography python-dbus
  python-enum34 python-gi python-idna python-ipaddress python-keyring
  python-keyrings.alt python-pip-

In [8]:
! git clone https://github.com/coccoc/coccoc-tokenizer
! pip3 install Cython 
% cd coccoc-tokenizer
! mkdir build
% cd build
! cmake -DBUILD_PYTHON=1 ..
! make install
% cd ../
% cd python
! python setup.py install

Cloning into 'coccoc-tokenizer'...
remote: Enumerating objects: 336, done.
remote: Total 336 (delta 0), reused 0 (delta 0), pack-reused 336
Receiving objects: 100% (336/336), 55.94 MiB | 20.33 MiB/s, done.
Resolving deltas: 100% (147/147), done.
/content/coccoc-tokenizer
/content/coccoc-tokenizer/build
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done
-- Generating done
-- Build files have been written to: /content/coccoc-tokenizer/

In [9]:
from CocCocTokenizer import PyTokenizer
T = PyTokenizer(load_nontone_data=True)

In [10]:
for category in dataset:
  for i in range(len(dataset[category])):
    corpus = dataset[category][i]
    corpus = T.word_tokenize(corpus)
    dataset[category][i] = corpus

### Loại bỏ stopword

Chúng ta sẽ sử dụng từ điểm stopwords có sẵn [tại đây](https://github.com/stopwords/vietnamese-stopwords/blob/master/vietnamese-stopwords.txt) để xử lý. 

Tiến hành download và đọc.

In [11]:
% cd ../../

/content


In [12]:
! wget https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords.txt 

--2020-09-10 12:54:21--  https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20475 (20K) [text/plain]
Saving to: ‘vietnamese-stopwords.txt’

vietnamese-stopword 100%[===================>]  20.00K  --.-KB/s    in 0.004s  

2020-09-10 12:54:21 (5.49 MB/s) - ‘vietnamese-stopwords.txt’ saved [20475/20475]



In [13]:
with open('/content/vietnamese-stopwords.txt', 'r') as f:
  lines = f.readlines()

stopwords = list()
for line in lines:
  stopwords.append(line.strip())

In [14]:
len(dataset['Doi song'][0])

1540

In [15]:
for category in dataset:
  for i in range(len(dataset[category])):
    corpus = dataset[category][i]
    tokens = list(token.lower() for token in corpus if token.replace('_', ' ').lower() not in stopwords)
    dataset[category][i] = tokens

In [16]:
len(dataset['Doi song'][0])

790

### Xây dựng từ điển cho dataset

In [17]:
vntc_dict = dict()

for category in dataset:
  for i in range(len(dataset[category])):
    for token in dataset[category][i]:
      try:
        vntc_dict[token] += 1
      except:
        vntc_dict[token] = 1

len_dict = len(vntc_dict)

In [18]:

vntc = {k: v for k, v in sorted(vntc_dict.items(), key=lambda item: item[1], reverse=True)}
vntc_dict = dict()

vntc_dict['<VOO>'] = 1
for i, key in enumerate(vntc):
  print(key)
  vntc_dict[key] = i + 2


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
nhưđài
loanlệnh
subtype
matti
taneli
daewo
xuân_lạc
autuần
đẹphọc
aukỳ
lịchhọc
dựnghọc
tínhhoá
uftaa
bổngcó
giao_an
loughborough
tuyđể
côngcông
tầng_bậc
icdadmahidolacth
hànghại
nổdo
thptnăm
xechất
điệp_trùng
philavông
poỏng
khỏn
nouhak
phoumsavan
souphanouvong
tướp
mắtngoảnh
kháchgiờ
warnier
đượcnhà
pkhuê
softtech
pmc
bilfinger
đỏffffff
loạch
xoạnh
đloan
vphong
gàchủ
ovbclub
gạnh
thui_thui
dẻo_quẹo
hìnhnhiều
nhụ
buýtngày
rù_rù
metrostroy
ngườigiờhướng
riverpark
trươngđình
acùng
trekking
phù_lá
hiệnngười
lịchđà
phápnhững
toànvnexpress
nhânthẻ
só
zon
vdv
hảiminh
wwwcicgcca
gree
đồngkiểu
cụm_trưởng
usdussdnăm
điểmvăn
louvieaux
aquitaine
parolin
saigontourit
honoluluhawaii
hanama
chănsavanakhet
luỗng
tiểu_thừa
cayxon
patuxay
phùm
hazmat
osha
gospodarski
biorecovery
kiếnchỉ
viênthời
thilịch
dcông
dashravdan
chimidtseren
badraa
dolgor
luvsanjav
atsolmon
phòngđt
cỡnhưng
bộntuổi
wwwcleverlearncom
wellstream
wheatley
alanwheatleysunderla

### Vector hóa từng corpus

In [19]:
max_len = 0

for category in dataset:
  for i in range(len(dataset[category])):
    if max_len < len(dataset[category][i]):
      max_len = len(dataset[category][i])


In [20]:
data = list()
labels = list()

In [21]:
for idx, category in enumerate(dataset):
  for i in range(len(dataset[category])):
    sent = list()
    for index, token in enumerate(dataset[category][i]):
      sent.append(vntc_dict[token])
    for j in range(index + 1, max_len):
      sent.append(1)
    data.append(sent)
    labels.append(idx)

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, random_state=42)

### Xây dựng mô hình phân loại text

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, LSTM, Bidirectional

In [25]:
bidrect_model = Sequential()
bidrect_model.add(Embedding(max_len, 512))
bidrect_model.add(Bidirectional(LSTM(32)))
bidrect_model.add(Dense(128, activation='relu'))
bidrect_model.add(Dense(10, activation='sigmoid'))

In [26]:
! nvidia-smi

Thu Sep 10 12:59:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    33W / 250W |    363MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [29]:
bidrect_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='acc')

In [ ]:
epochs = 10 
batch_size = 16
bidrect_model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test,y_test), batch_size=batch_size, verbose=1, shuffle=False)